In [1]:
import io
import logging
import numpy as np
import os
import pickle
import typing
from tqdm import tqdm
from typing import Any, Dict, List, Optional, Text, Tuple

from rasa_nlu.classifiers import INTENT_RANKING_LENGTH
from rasa_nlu.components import Component

logger = logging.getLogger(__name__)

if typing.TYPE_CHECKING:
    import tensorflow as tf
    from rasa_nlu.config import RasaNLUModelConfig
    from rasa_nlu.training_data import TrainingData
    from rasa_nlu.model import Metadata
    from rasa_nlu.training_data import Message

try:
    import tensorflow as tf
except ImportError:
    tf = None

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [259]:
class EmbeddingIntentClassifier(Component):
    """Intent classifier using supervised embeddings.
    The embedding intent classifier embeds user inputs
    and intent labels into the same space.
    Supervised embeddings are trained by maximizing similarity between them.
    It also provides rankings of the labels that did not "win".
    The embedding intent classifier needs to be preceded by
    a featurizer in the pipeline.
    This featurizer creates the features used for the embeddings.
    It is recommended to use ``CountVectorsFeaturizer`` that
    can be optionally preceded by ``SpacyNLP`` and ``SpacyTokenizer``.
    Based on the starspace idea from: https://arxiv.org/abs/1709.03856.
    However, in this implementation the `mu` parameter is treated differently
    and additional hidden layers are added together with dropout.
    """

    provides = ["intent", "intent_ranking"]

    requires = ["text_features"]

    defaults = {
        # nn architecture
        # sizes of hidden layers before the embedding layer for input words
        # the number of hidden layers is thus equal to the length of this list
        "hidden_layers_sizes_a": [256, 128],
        # sizes of hidden layers before the embedding layer for intent labels
        # the number of hidden layers is thus equal to the length of this list
        "hidden_layers_sizes_b": [],

        # training parameters
        # initial and final batch sizes - batch size will be
        # linearly increased for each epoch
        "batch_size": [64, 256],
        # number of epochs
        "epochs": 30,

        # embedding parameters
        # dimension size of embedding vectors
        "embed_dim": 20,
        # how similar the algorithm should try
        # to make embedding vectors for correct intent labels
        "mu_pos": 0.8,  # should be 0.0 < ... < 1.0 for 'cosine'
        # maximum negative similarity for incorrect intent labels
        "mu_neg": -0.4,  # should be -1.0 < ... < 1.0 for 'cosine'
        # the type of the similarity
        "similarity_type": 'cosine',  # string 'cosine' or 'inner'
        # the number of incorrect intents, the algorithm will minimize
        # their similarity to the input words during training
        "num_neg": 20,
        # flag: if true, only minimize the maximum similarity for
        # incorrect intent labels
        "use_max_sim_neg": True,
        # set random seed to any int to get reproducible results
        # try to change to another int if you are not getting good results
        "random_seed": 1,

        # regularization parameters
        # the scale of L2 regularization
        "C2": 0.002,
        # the scale of how critical the algorithm should be of minimizing the
        # maximum similarity between embeddings of different intent labels
        "C_emb": 0.8,
        # dropout rate for rnn
        "droprate": 0.2,

        # flag: if true, the algorithm will split the intent labels into tokens
        #       and use bag-of-words representations for them
        "intent_tokenization_flag": False,
        # delimiter string to split the intent labels
        "intent_split_symbol": '_',

        # visualization of accuracy
        # how often to calculate training accuracy
        "evaluate_every_num_epochs": 10,  # small values may hurt performance
        # how many examples to use for calculation of training accuracy
        "evaluate_on_num_examples": 1000  # large values may hurt performance
    }

    def __init__(self,
                 component_config: Optional[Dict[Text, Any]] = None,
                 inv_intent_dict: Optional[Dict[int, Text]] = None,
                 encoded_all_intents: Optional[np.ndarray] = None,
                 session: Optional['tf.Session'] = None,
                 graph: Optional['tf.Graph'] = None,
                 message_placeholder: Optional['tf.Tensor'] = None,
                 intent_placeholder: Optional['tf.Tensor'] = None,
                 similarity_op: Optional['tf.Tensor'] = None,
                 word_embed: Optional['tf.Tensor'] = None,
                 intent_embed: Optional['tf.Tensor'] = None
                 ) -> None:
        """Declare instant variables with default values"""

        self._check_tensorflow()
        super(EmbeddingIntentClassifier, self).__init__(component_config)

        self._load_params()

        # transform numbers to intents
        self.inv_intent_dict = inv_intent_dict
        # encode all intents with numbers
        self.encoded_all_intents = encoded_all_intents

        # tf related instances
        self.session = session
        self.graph = graph
        self.a_in = message_placeholder
        self.b_in = intent_placeholder
        self.sim_op = similarity_op

        # persisted embeddings
        self.word_embed = word_embed
        self.intent_embed = intent_embed

    # init helpers
    def _load_nn_architecture_params(self, config: Dict[Text, Any]) -> None:
        self.hidden_layer_sizes = {'a': config['hidden_layers_sizes_a'],
                                   'b': config['hidden_layers_sizes_b']}

        self.batch_size = config['batch_size']
        self.epochs = config['epochs']

    def _load_embedding_params(self, config: Dict[Text, Any]) -> None:
        self.embed_dim = config['embed_dim']
        self.mu_pos = config['mu_pos']
        self.mu_neg = config['mu_neg']
        self.similarity_type = config['similarity_type']
        self.num_neg = config['num_neg']
        self.use_max_sim_neg = config['use_max_sim_neg']
        self.random_seed = self.component_config['random_seed']

    def _load_regularization_params(self, config: Dict[Text, Any]) -> None:
        self.C2 = config['C2']
        self.C_emb = config['C_emb']
        self.droprate = config['droprate']

    def _load_flag_if_tokenize_intents(self, config: Dict[Text, Any]) -> None:
        self.intent_tokenization_flag = config['intent_tokenization_flag']
        self.intent_split_symbol = config['intent_split_symbol']
        if self.intent_tokenization_flag and not self.intent_split_symbol:
            logger.warning("intent_split_symbol was not specified, "
                           "so intent tokenization will be ignored")
            self.intent_tokenization_flag = False

    def _load_visual_params(self, config: Dict[Text, Any]) -> None:
        self.evaluate_every_num_epochs = config['evaluate_every_num_epochs']
        if self.evaluate_every_num_epochs < 1:
            self.evaluate_every_num_epochs = self.epochs

        self.evaluate_on_num_examples = config['evaluate_on_num_examples']

    def _load_params(self) -> None:

        self._load_nn_architecture_params(self.component_config)
        self._load_embedding_params(self.component_config)
        self._load_regularization_params(self.component_config)
        self._load_flag_if_tokenize_intents(self.component_config)
        self._load_visual_params(self.component_config)

    # package safety checks
    @classmethod
    def required_packages(cls) -> List[Text]:
        return ["tensorflow"]

    @staticmethod
    def _check_tensorflow():
        if tf is None:
            raise ImportError(
                'Failed to import `tensorflow`. '
                'Please install `tensorflow`. '
                'For example with `pip install tensorflow`.')

    # training data helpers:
    @staticmethod
    def _create_intent_dict(training_data: 'TrainingData') -> Dict[Text, int]:
        """Create intent dictionary"""

        distinct_intents = set([example.get("intent")
                                for example in training_data.intent_examples])
        return {intent: idx
                for idx, intent in enumerate(sorted(distinct_intents))}

    @staticmethod
    def _create_intent_token_dict(intents: List[Text],
                                  intent_split_symbol: Text) -> Dict[Text, int]:
        """Create intent token dictionary"""

        distinct_tokens = set([token
                               for intent in intents
                               for token in intent.split(intent_split_symbol)])
        return {token: idx
                for idx, token in enumerate(sorted(distinct_tokens))}

    def _create_encoded_intents(self,
                                intent_dict: Dict[Text, int]) -> np.ndarray:
        """Create matrix with intents encoded in rows as bag of words.
        If intent_tokenization_flag is off, returns identity matrix.
        """

        if self.intent_tokenization_flag:
            intent_token_dict = self._create_intent_token_dict(
                list(intent_dict.keys()), self.intent_split_symbol)

            encoded_all_intents = np.zeros((len(intent_dict),
                                            len(intent_token_dict)))
            for key, idx in intent_dict.items():
                for t in key.split(self.intent_split_symbol):
                    encoded_all_intents[idx, intent_token_dict[t]] = 1

            return encoded_all_intents
        else:
            return np.eye(len(intent_dict))

    # noinspection PyPep8Naming
    def _create_all_Y(self, size: int) -> np.ndarray:
        """Stack encoded_all_intents on top of each other
        to create candidates for training examples and
        to calculate training accuracy
        """

        return np.stack([self.encoded_all_intents] * size)

    # noinspection PyPep8Naming
    def _prepare_data_for_training(
            self,
            training_data: 'TrainingData',
            intent_dict: Dict[Text, int]
    ) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        """Prepare data for training"""

        X = np.stack([e.get("text_features")
                      for e in training_data.intent_examples])

        intents_for_X = np.array([intent_dict[e.get("intent")]
                                  for e in training_data.intent_examples])

        Y = np.stack([self.encoded_all_intents[intent_idx]
                      for intent_idx in intents_for_X])

        return X, Y, intents_for_X

    # tf helpers:
    def _create_tf_embed_nn(self, x_in: 'tf.Tensor', is_training: 'tf.Tensor',
                            layer_sizes: List[int], name: Text) -> 'tf.Tensor':
        """Create nn with hidden layers and name"""

        reg = tf.contrib.layers.l2_regularizer(self.C2)
        x = x_in
        for i, layer_size in enumerate(layer_sizes):
            x = tf.layers.dense(inputs=x,
                                units=layer_size,
                                activation=tf.nn.relu,
                                kernel_regularizer=reg,
                                name='hidden_layer_{}_{}'.format(name, i))
            x = tf.layers.dropout(x, rate=self.droprate, training=is_training)

        x = tf.layers.dense(inputs=x,
                            units=self.embed_dim,
                            kernel_regularizer=reg,
                            name='embed_layer_{}'.format(name))
        return x

    def _create_tf_embed(self,
                         a_in: 'tf.Tensor',
                         b_in: 'tf.Tensor',
                         is_training: 'tf.Tensor'
                         ) -> Tuple['tf.Tensor', 'tf.Tensor']:
        """Create tf graph for training"""

        emb_a = self._create_tf_embed_nn(a_in, is_training,
                                         self.hidden_layer_sizes['a'],
                                         name='a')
        emb_b = self._create_tf_embed_nn(b_in, is_training,
                                         self.hidden_layer_sizes['b'],
                                         name='b')
        return emb_a, emb_b

    def _tf_sim(self,
                a: 'tf.Tensor',
                b: 'tf.Tensor') -> Tuple['tf.Tensor', 'tf.Tensor']:
        """Define similarity
        in two cases:
            sim: between embedded words and embedded intent labels
            sim_emb: between individual embedded intent labels only
        """

        if self.similarity_type == 'cosine':
            # normalize embedding vectors for cosine similarity
            a = tf.nn.l2_normalize(a, -1)
            b = tf.nn.l2_normalize(b, -1)

        if self.similarity_type in {'cosine', 'inner'}:
            sim = tf.reduce_sum(tf.expand_dims(a, 1) * b, -1)
            sim_emb = tf.reduce_sum(b[:, 0:1, :] * b[:, 1:, :], -1)

            return sim, sim_emb

        else:
            raise ValueError("Wrong similarity type {}, "
                             "should be 'cosine' or 'inner'"
                             "".format(self.similarity_type))

    def _tf_loss(self, sim: 'tf.Tensor', sim_emb: 'tf.Tensor') -> 'tf.Tensor':
        """Define loss"""

        # loss for maximizing similarity with correct action
        loss = tf.maximum(0., self.mu_pos - sim[:, 0])

        if self.use_max_sim_neg:
            # minimize only maximum similarity over incorrect actions
            max_sim_neg = tf.reduce_max(sim[:, 1:], -1)
            loss += tf.maximum(0., self.mu_neg + max_sim_neg)
        else:
            # minimize all similarities with incorrect actions
            max_margin = tf.maximum(0., self.mu_neg + sim[:, 1:])
            loss += tf.reduce_sum(max_margin, -1)

        # penalize max similarity between intent embeddings
        max_sim_emb = tf.maximum(0., tf.reduce_max(sim_emb, -1))
        loss += max_sim_emb * self.C_emb

        # average the loss over the batch and add regularization losses
        loss = (tf.reduce_mean(loss) + tf.losses.get_regularization_loss())
        return loss

    # training helpers:
    def _create_batch_b(self, batch_pos_b: np.ndarray,
                        intent_ids: np.ndarray) -> np.ndarray:
        """Create batch of intents.
        Where the first is correct intent
        and the rest are wrong intents sampled randomly
        """

        batch_pos_b = batch_pos_b[:, np.newaxis, :]

        # sample negatives
        batch_neg_b = np.zeros((batch_pos_b.shape[0], self.num_neg,
                                batch_pos_b.shape[-1]))
        for b in range(batch_pos_b.shape[0]):
            # create negative indexes out of possible ones
            # except for correct index of b
            negative_indexes = [i for i in
                                range(self.encoded_all_intents.shape[0])
                                if i != intent_ids[b]]
            negs = np.random.choice(negative_indexes, size=self.num_neg)

            batch_neg_b[b] = self.encoded_all_intents[negs]

        return np.concatenate([batch_pos_b, batch_neg_b], 1)

    def _linearly_increasing_batch_size(self, epoch: int) -> int:
        """Linearly increase batch size with every epoch.
        The idea comes from https://arxiv.org/abs/1711.00489
        """

        if not isinstance(self.batch_size, list):
            return int(self.batch_size)

        if self.epochs > 1:
            return int(self.batch_size[0] +
                       epoch * (self.batch_size[1] -
                                self.batch_size[0]) / (self.epochs - 1))
        else:
            return int(self.batch_size[0])

    # noinspection PyPep8Naming
    def _train_tf(self,
                  X: np.ndarray,
                  Y: np.ndarray,
                  intents_for_X: np.ndarray,
                  loss: 'tf.Tensor',
                  is_training: 'tf.Tensor',
                  train_op: 'tf.Tensor'
                  ) -> None:
        """Train tf graph"""

        self.session.run(tf.global_variables_initializer())

        if self.evaluate_on_num_examples:
            logger.info("Accuracy is updated every {} epochs"
                        "".format(self.evaluate_every_num_epochs))

        pbar = tqdm(range(self.epochs), desc="Epochs")
        train_acc = 0
        last_loss = 0
        for ep in pbar:
            indices = np.random.permutation(len(X))

            batch_size = self._linearly_increasing_batch_size(ep)
            batches_per_epoch = (len(X) // batch_size +
                                 int(len(X) % batch_size > 0))

            ep_loss = 0
            for i in range(batches_per_epoch):
                end_idx = (i + 1) * batch_size
                start_idx = i * batch_size
                batch_a = X[indices[start_idx:end_idx]]
                batch_pos_b = Y[indices[start_idx:end_idx]]
                intents_for_b = intents_for_X[indices[start_idx:end_idx]]
                # add negatives
                batch_b = self._create_batch_b(batch_pos_b, intents_for_b)

                sess_out = self.session.run(
                    {'loss': loss, 'train_op': train_op},
                    feed_dict={self.a_in: batch_a,
                               self.b_in: batch_b,
                               is_training: True}
                )
                ep_loss += sess_out.get('loss') / batches_per_epoch

            if self.evaluate_on_num_examples:
                if (ep == 0 or
                        (ep + 1) % self.evaluate_every_num_epochs == 0 or
                        (ep + 1) == self.epochs):
                    train_acc = self._output_training_stat(X, intents_for_X,
                                                           is_training)
                    last_loss = ep_loss

                pbar.set_postfix({
                    "loss": "{:.3f}".format(ep_loss),
                    "acc": "{:.3f}".format(train_acc)
                })
            else:
                pbar.set_postfix({
                    "loss": "{:.3f}".format(ep_loss)
                })

        if self.evaluate_on_num_examples:
            logger.info("Finished training embedding classifier, "
                        "loss={:.3f}, train accuracy={:.3f}"
                        "".format(last_loss, train_acc))

    # noinspection PyPep8Naming
    def _output_training_stat(self,
                              X: np.ndarray,
                              intents_for_X: np.ndarray,
                              is_training: 'tf.Tensor') -> np.ndarray:
        """Output training statistics"""

        n = self.evaluate_on_num_examples
        ids = np.random.permutation(len(X))[:n]
        all_Y = self._create_all_Y(X[ids].shape[0])

        train_sim = self.session.run(self.sim_op,
                                     feed_dict={self.a_in: X[ids],
                                                self.b_in: all_Y,
                                                is_training: False})

        train_acc = np.mean(np.argmax(train_sim, -1) == intents_for_X[ids])
        return train_acc

    def train(self,
              training_data: 'TrainingData',
              cfg: Optional['RasaNLUModelConfig'] = None,
              **kwargs: Any) -> None:
        """Train the embedding intent classifier on a data set."""

        intent_dict = self._create_intent_dict(training_data)
        if len(intent_dict) < 2:
            logger.error("Can not train an intent classifier. "
                         "Need at least 2 different classes. "
                         "Skipping training of intent classifier.")
            return

        self.inv_intent_dict = {v: k for k, v in intent_dict.items()}
        self.encoded_all_intents = self._create_encoded_intents(
            intent_dict)

        # noinspection PyPep8Naming
        X, Y, intents_for_X = self._prepare_data_for_training(
            training_data, intent_dict)

        # check if number of negatives is less than number of intents
        logger.debug("Check if num_neg {} is smaller than "
                     "number of intents {}, "
                     "else set num_neg to the number of intents - 1"
                     "".format(self.num_neg,
                               self.encoded_all_intents.shape[0]))
        self.num_neg = min(self.num_neg,
                           self.encoded_all_intents.shape[0] - 1)

        self.graph = tf.Graph()
        with self.graph.as_default():
            # set random seed
            np.random.seed(self.random_seed)
            tf.set_random_seed(self.random_seed)

            self.a_in = tf.placeholder(tf.float32, (None, X.shape[-1]),
                                       name='a')
            self.b_in = tf.placeholder(tf.float32, (None, None, Y.shape[-1]),
                                       name='b')

            is_training = tf.placeholder_with_default(False, shape=())

            (self.word_embed,
             self.intent_embed) = self._create_tf_embed(self.a_in, self.b_in,
                                                        is_training)

            self.sim_op, sim_emb = self._tf_sim(self.word_embed,
                                                self.intent_embed)
            loss = self._tf_loss(self.sim_op, sim_emb)

            train_op = tf.train.AdamOptimizer().minimize(loss)

            # train tensorflow graph
            self.session = tf.Session()

            self._train_tf(X, Y, intents_for_X,
                           loss, is_training, train_op)

    # process helpers
    # noinspection PyPep8Naming
    def _calculate_message_sim(self,
                               X: np.ndarray,
                               all_Y: np.ndarray
                               ) -> Tuple[np.ndarray, List[float]]:
        """Load tf graph and calculate message similarities"""

        message_sim = self.session.run(self.sim_op,
                                       feed_dict={self.a_in: X,
                                                  self.b_in: all_Y})
        message_sim = message_sim.flatten()  # sim is a matrix

        intent_ids = message_sim.argsort()[::-1]
        message_sim[::-1].sort()

        if self.similarity_type == 'cosine':
            # clip negative values to zero
            message_sim[message_sim < 0] = 0
        elif self.similarity_type == 'inner':
            # normalize result to [0, 1] with softmax
            message_sim = np.exp(message_sim)
            message_sim /= np.sum(message_sim)

        # transform sim to python list for JSON serializing
        return intent_ids, message_sim.tolist()

    def process(self, message: 'Message', **kwargs: Any) -> None:
        """Return the most likely intent and its similarity to the input."""

        intent = {"name": None, "confidence": 0.0}
        intent_ranking = []

        if self.session is None:
            logger.error("There is no trained tf.session: "
                         "component is either not trained or "
                         "didn't receive enough training data")

        else:
            # get features (bag of words) for a message
            # noinspection PyPep8Naming
            X = message.get("text_features").reshape(1, -1)

            # stack encoded_all_intents on top of each other
            # to create candidates for test examples
            # noinspection PyPep8Naming
            all_Y = self._create_all_Y(X.shape[0])

            # load tf graph and session
            intent_ids, message_sim = self._calculate_message_sim(X, all_Y)

            # if X contains all zeros do not predict some label
            if X.any() and intent_ids.size > 0:
                intent = {"name": self.inv_intent_dict[intent_ids[0]],
                          "confidence": message_sim[0]}

                ranking = list(zip(list(intent_ids), message_sim))
                ranking = ranking[:INTENT_RANKING_LENGTH]
                intent_ranking = [{"name": self.inv_intent_dict[intent_idx],
                                   "confidence": score}
                                  for intent_idx, score in ranking]

        message.set("intent", intent, add_to_output=True)
        message.set("intent_ranking", intent_ranking, add_to_output=True)

    def persist(self,
                file_name: Text,
                model_dir: Text) -> Dict[Text, Any]:
        """Persist this model into the passed directory.
        Return the metadata necessary to load the model again.
        """

        if self.session is None:
            return {"file": None}

        checkpoint = os.path.join(model_dir, file_name + ".ckpt")

        try:
            os.makedirs(os.path.dirname(checkpoint))
        except OSError as e:
            # be happy if someone already created the path
            import errno
            if e.errno != errno.EEXIST:
                raise
        with self.graph.as_default():
            self.graph.clear_collection('message_placeholder')
            self.graph.add_to_collection('message_placeholder',
                                         self.a_in)

            self.graph.clear_collection('intent_placeholder')
            self.graph.add_to_collection('intent_placeholder',
                                         self.b_in)

            self.graph.clear_collection('similarity_op')
            self.graph.add_to_collection('similarity_op',
                                         self.sim_op)

            self.graph.clear_collection('word_embed')
            self.graph.add_to_collection('word_embed',
                                         self.word_embed)
            self.graph.clear_collection('intent_embed')
            self.graph.add_to_collection('intent_embed',
                                         self.intent_embed)

            saver = tf.train.Saver()
            saver.save(self.session, checkpoint)

        with io.open(os.path.join(
                model_dir,
                file_name + "_inv_intent_dict.pkl"), 'wb') as f:
            pickle.dump(self.inv_intent_dict, f)
        with io.open(os.path.join(
                model_dir,
                file_name + "_encoded_all_intents.pkl"), 'wb') as f:
            pickle.dump(self.encoded_all_intents, f)

        return {"file": file_name}

    @classmethod
    def load(cls,
             meta: Dict[Text, Any],
             model_dir: Text = None,
             model_metadata: 'Metadata' = None,
             cached_component: Optional['EmbeddingIntentClassifier'] = None,
             **kwargs: Any
             ) -> 'EmbeddingIntentClassifier':

        if model_dir and meta.get("file"):
            file_name = meta.get("file")
            checkpoint = os.path.join(model_dir, file_name + ".ckpt")
            graph = tf.Graph()
            with graph.as_default():
                sess = tf.Session()
                saver = tf.train.import_meta_graph(checkpoint + '.meta')

                saver.restore(sess, checkpoint)

                a_in = tf.get_collection('message_placeholder')[0]
                b_in = tf.get_collection('intent_placeholder')[0]

                sim_op = tf.get_collection('similarity_op')[0]

                word_embed = tf.get_collection('word_embed')[0]
                intent_embed = tf.get_collection('intent_embed')[0]

            with io.open(os.path.join(
                    model_dir,
                    file_name + "_inv_intent_dict.pkl"), 'rb') as f:
                inv_intent_dict = pickle.load(f)
            with io.open(os.path.join(
                    model_dir,
                    file_name + "_encoded_all_intents.pkl"), 'rb') as f:
                encoded_all_intents = pickle.load(f)

            return cls(
                component_config=meta,
                inv_intent_dict=inv_intent_dict,
                encoded_all_intents=encoded_all_intents,
                session=sess,
                graph=graph,
                message_placeholder=a_in,
                intent_placeholder=b_in,
                similarity_op=sim_op,
                word_embed=word_embed,
                intent_embed=intent_embed
            )

        else:
            logger.warning("Failed to load nlu model. Maybe path {} "
                           "doesn't exist"
                           "".format(os.path.abspath(model_dir)))
            return cls(component_config=meta)

In [230]:
from rasa_nlu.training_data import TrainingData, Message
import spacy
spacy_nlp = spacy.load('en')
examples = [Message("anywhere in the west", {
                    "intent": "restaurant_search",
                    "entities": [{"start": 16, "end": 20,
                                  "value": "west", "entity": "location"}],
                    "spacy_doc": spacy_nlp("anywhere in the west")
                    }),
            Message("central indian restaurant", {
                    "intent": "restaurant_search",
                    "entities": [
                     {"start": 0, "end": 7, "value": "central",
                      "entity": "location",
                      "extractor": "random_extractor"},
                     {"start": 8, "end": 14, "value": "indian",
                      "entity": "cuisine",
                      "extractor": "CRFEntityExtractor"}
                                 ],
                    "spacy_doc": spacy_nlp("central indian restaurant")
                    }),
            Message("hi there!", {"intent": "greet", "entities": [],
                                  "spacy_doc": spacy_nlp("hi there!")}),
            Message("good morning", {"intent": "greet", "entities": [],         
                                     "spacy_doc": spacy_nlp("good morning")}),
            Message("thank you", {"intent": "thanks", "entities": [],
                                  "spacy_doc": spacy_nlp("thank you")}),
            Message("good bye", {"intent": "thanks", "entities": [],
                                 "spacy_doc": spacy_nlp("good bye")})        
      ]

In [231]:
training_data = TrainingData(training_examples=examples)

/anaconda3/lib/python3.6/site-packages/rasa_nlu/training_data/training_data.py:194: UserWarning: Entity 'cuisine' has only 1 training examples! minimum is 2, training may fail.
  self.MIN_EXAMPLES_PER_ENTITY))


In [232]:
from rasa_nlu.featurizers.count_vectors_featurizer import CountVectorsFeaturizer
ftr = CountVectorsFeaturizer({"token_pattern": r'(?u)\b\w+\b'})
ftr.train(training_data)

In [233]:
classifier = EmbeddingIntentClassifier()

In [234]:
classifier.train(training_data=training_data)

Epochs: 100%|██████████| 300/300 [00:01<00:00, 209.18it/s, loss=0.096, acc=1.000]


In [100]:
test_inp = Message("show me some indian restuarants")

In [101]:
test_inp.as_dict()

{'text': 'show me some indian restuarants'}

In [102]:
ftr.process(test_inp)

In [103]:
test_inp.as_dict()

{'text_features': array([[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]]),
 'text': 'show me some indian restuarants'}

In [104]:
classifier.process(test_inp)

In [105]:
test_inp.as_dict()

{'text_features': array([[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]]),
 'intent': {'name': 'restaurant_search', 'confidence': 0.8687058687210083},
 'intent_ranking': [{'name': 'restaurant_search',
   'confidence': 0.8687058687210083},
  {'name': 'greet', 'confidence': 0.35430455207824707},
  {'name': 'thanks', 'confidence': 0.0}],
 'text': 'show me some indian restuarants'}

# train model using json input

In [260]:
inp_data = {
  "intent":{
   "restaurant_search": ["anywhere in the west", "central indian restaurant", "looking for french hotels",
						"show me some south indian places to eat in the north", "looking for exotic eating spots",
						"any north indian hotels near me", "show me chinese restaurants", "want to go to an italian restaurant",
						"i'm looking for a place in the north of town", "show me a mexican place in the centre",
						"hi, show me some authentic tamilian hotels", "hello, i need some fish food", "good morning, i'm looking for spanish or mexican hotels, thanks"],
   "greet": ["hi", "hello", "good day", "hi!", "hey", "good morning", "good evening", "hey there!", "hey, how are you?"],
   "thanks": ["thanks", "thank you", "thanx", "this is helpful, thanks", "regards", "good bye", "bye", "thanks & regards", "thanks and regards",
             "with regards"]
  },
 "entities": {
  "restaurant_search": [[{"value": "west", "entity": "location"}], [{"value": "central", "entity": "location"}, {"value": "indian", "entity": "cuisine"}], [{"value": "french", "entity": "cuisine"}], [{"value": "south indian", "entity": "cuisine"}, {"value": "north", "entity": "location"}], [], [{"value": "north indian", "entity": "cuisine"}], [{"value": "chinese", "entity": "cuisine"}], [{"value": "italian", "entity": "cuisine"}], [{"value": "north", "entity": "location"}], [{"value": "mexican", "entity": "cuisine"}, {"value": "centre", "entity": "location"}], [{"value": "tamilian", "entity": "cuisine"}], [{"value": "fish", "entity": "cuisine"}], [{"value": "spanish", "entity": "cuisine"}, {"value": "mexican", "entity": "cuisine"}]],
  "greet": [[], [], [], [], [], [], [], [], []],
  "thanks": [[], [], [], [], [], [], [], [], [], []]
 }
}

In [261]:
CONTRACTIONS = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "i had / i would",
"i'd've": "i would have",
"i'll": "i shall / i will",
"i'll've": "i shall have / i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [262]:
# preprocessing input data
# lowering, contractions, regex to remove special characters
import re
def regex_cleaning(text):
    text = text.lower()
    text = text.replace('\x01', ' ')
    text = re.sub(r'[^\x00-\x7F]+',' ', text)
    text = re.sub(r"http[s]?\S+", " ", text)
    text = re.sub(r"xx+", " ", text)
    text = re.sub(r"x{2,}", "", text)
    text = re.sub(r"www\S+\.com", "", text)
    text = re.sub(r"\S+fresh[a-zA-Z]+.com", " ", text)
    text = re.sub(r"\S+@\S+", "", text)
    text = re.sub(r"[-()._\#/@&%;*:<>{}+=~!|?,]", " ", text)
    text = re.sub(r"[\[\]]", "", text)
    text = re.sub(r"`", "", text)
    text = text.replace('"', '')
    text = text.replace("'", "")
    return text

def preprocess(text):
    # lower
    text = str(text).lower()
    # contractions
    text = " ".join(CONTRACTIONS[y.lower()].split('/')[0]
                    if y.lower() in CONTRACTIONS else y
                    for y in str(text).split())
    # regex cleaning
    text = regex_cleaning(text)
    return text
    
inp_data_pre = {}
for key, value in inp_data.items():
    print(key)
    inp_data_pre[key] = {}
    for key1, value1 in value.items():
        print(key1)
        if key == "intent":
            value1_new = [preprocess(x) for x in value1]
        elif key == "entities":
            value1_new = []
            for item in value1:
                new_item = []
                for item1 in item:
                    if type(item1['value']) == str:
                        item1['value'] = preprocess(item1['value'])
                    new_item.append(item1)
                value1_new.append(new_item)
        inp_data_pre[key][key1] = value1_new

intent
restaurant_search
greet
thanks
entities
restaurant_search
greet
thanks


In [263]:
inp_data_pre

{'intent': {'restaurant_search': ['anywhere in the west',
   'central indian restaurant',
   'looking for french hotels',
   'show me some south indian places to eat in the north',
   'looking for exotic eating spots',
   'any north indian hotels near me',
   'show me chinese restaurants',
   'want to go to an italian restaurant',
   'i am looking for a place in the north of town',
   'show me a mexican place in the centre',
   'hi  show me some authentic tamilian hotels',
   'hello  i need some fish food',
   'good morning  i am looking for spanish or mexican hotels  thanks'],
  'greet': ['hi',
   'hello',
   'good day',
   'hi ',
   'hey',
   'good morning',
   'good evening',
   'hey there ',
   'hey  how are you '],
  'thanks': ['thanks',
   'thank you',
   'thanx',
   'this is helpful  thanks',
   'regards',
   'good bye',
   'bye',
   'thanks   regards',
   'thanks and regards',
   'with regards']},
 'entities': {'restaurant_search': [[{'value': 'west', 'entity': 'location'}],
  

In [264]:
from rasa_nlu.training_data import TrainingData, Message
import spacy
spacy_nlp = spacy.load('en')

In [265]:
# prepare inp_data for training
intents = inp_data_pre['intent']
entities = inp_data_pre['entities']

In [266]:
def get_start_end_pos_entity(utterance, entity):
    start_pos = utterance.index(entity)
    end_pos = start_pos + len(entity)
    return start_pos, end_pos

In [267]:
examples = []
for intent, intent_values in intents.items():
    print(intent)
    entity_values = entities[intent]
    for i, intent_value in enumerate(intent_values):
        out = {}
        entity_value = entity_values[i]
        if entity_value:
            entity_value_new = []
            for ent in entity_value:
                start_pos, end_pos = get_start_end_pos_entity(intent_value,
                                                              ent['value'])
                ent['start'] = start_pos
                ent['end'] = end_pos
                entity_value_new.append(ent)
        else:
            entity_value_new = entity_value
        # update out dict
        out['intent'] = intent
        out['entities'] = entity_value_new
        out['spacy_doc']= spacy_nlp(intent_value)
        examples.append(Message(intent_value, out))

restaurant_search
greet
thanks


In [276]:
training_data = TrainingData(training_examples=examples)

In [277]:
len(training_data.intent_examples) == len(examples)

True

In [278]:
from rasa_nlu.featurizers.count_vectors_featurizer import CountVectorsFeaturizer
ftr = CountVectorsFeaturizer({"token_pattern": r'(?u)\b\w+\b'})
ftr.train(training_data)

In [279]:
classifier = EmbeddingIntentClassifier()

In [280]:
classifier.train(training_data=training_data)

Epochs: 100%|██████████| 30/30 [00:00<00:00, 65.45it/s, loss=0.285, acc=1.000]


In [281]:
# save
classifier.persist(file_name="embedding_intent_classifier_v1", model_dir="/Users/varunn/Documents/projects/Intent_and_Entity_Extraction/models/")
#import pickle
#pickle.dump(ftr, open('/Users/varunn/Documents/projects/Intent_and_Entity_Extraction/models/count_vectorizer_v1.pkl', 'wb'))

{'file': 'embedding_intent_classifier_v1'}

In [290]:
test_inp = "hi, looking for chinese hotels"
test_inp_pre = preprocess(test_inp)
test_inp_pre_m = Message(test_inp_pre)
ftr.process(test_inp_pre_m)
classifier.process(test_inp_pre_m)

In [291]:
test_inp_pre_m.as_dict()['intent']

{'name': 'restaurant_search', 'confidence': 0.7194852828979492}

In [194]:
classifier.persist(file_name="embedding_intent_classifier_v1", model_dir="/Users/varunn/Documents/projects/Intent_and_Entity_Extraction/models/")

{'file': 'embedding_intent_classifier_v1'}

In [204]:
!pip3 install pickle
import pickle
pickle.dump(ftr, open('/Users/varunn/Documents/projects/Intent_and_Entity_Extraction/models/count_vectorizer_v1.pkl', 'w'))

  Could not find a version that satisfies the requirement pickle (from versions: )
No matching distribution found for pickle
You are using pip version 18.0, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


AttributeError: Can't pickle local object 'CountVectorsFeaturizer.__init__.<locals>.<lambda>'

In [133]:
meta = {'file': 'temp_starspace_model'}
classifier1 = EmbeddingIntentClassifier.load(meta=meta, model_dir='./')

OSError: File ./temp_starspace_model.ckpt.meta does not exist.

In [143]:
test_inp = "looking for chinese hotels"
test_inp_pre = preprocess(test_inp)
test_inp_pre_m = Message(test_inp_pre)
ftr.process(test_inp_pre_m)
classifier1.process(test_inp_pre_m)

In [144]:
test_inp_pre_m.as_dict()['intent']

{'text_features': array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 'intent': {'name': 'restaurant_search', 'confidence': 0.8807507157325745},
 'intent_ranking': [{'name': 'restaurant_search',
   'confidence': 0.8807507157325745},
  {'name': 'thanks', 'confidence': 0.17054714262485504},
  {'name': 'greet', 'confidence': 0.0}],
 'text': 'looking for chinese hotels'}